In [1]:
import pandas as pd
import os
import openpyxl as px
import re 

In [2]:
cwd = os.getcwd()
datadir = '/'.join(cwd.split('/')[0:-1]) + '/Raw Data/'
df = pd.read_excel(datadir+'/judges.xlsx', sheetname="judges")
df['law school']= df['law school'].fillna('Unknown')
df['re-elect years'] = df['re-elect years'].fillna('None')

In [3]:
df['law school'] = df['law school'].apply(lambda x:re.sub(' ','',x.strip()).lower())
#Get distinct name list 
list_law_school = df['law school'].unique()

In [4]:
list_electyear = df['re-elect years'].unique()

In [6]:
list_electyear

array(['None', '1993; 2007', 2010, 2004, '2003, 2011', 2005, 'x', 2009,
       2008, 2013, 2014, 2007, '1997, 2011', 2011, 2012, '1994, 2004',
       '2001, 2008', '2013?', '2000, 2015', '2002, 2012', '2000, 2010',
       '2002, 2011', '1992, 2006', '1995, 2002, 2007, 2013', 2015,
       '1999, 2009', '2012?', '1984; 1994?', '2004, 2007', '2009, 2013',
       '1994; 2004', 1995, '1989; 1999', '2004, 2014', '1996, 2006',
       '1993; 2003', 1985, 2003, '2002, 2016', 2002, '1995, 1998, 2012',
       '1990, 2004', 2006, '1988, 1998, 2008', '2001, 2011',
       '1981, 1986, 2000', '1988, 2002, 2016', '1989, 2003',
       '1990; 2005; 2011', '2001; 2011', '2007, 2014', '2000;', 1999, 2000,
       '1989, 1999', '1997; 2007', '2002, 2007, 2014', '1994, 2004, 2014',
       '1999; 2009', '1991, 2001, 2011', '2006?', '2006, 2013',
       '1995; 2009', '1988; 2003', '1999; 2013', '2007; 2013',
       '1999; 2010', '2004; 2007', '1993, 2007', '2003; 2009; 2014', 1998,
       '1976; 1980', '1999?'

In [7]:
list_electyear_unique = []
for e in list_electyear:
    if ';' in str(e):
        c = e.count(';')
        for each in e.split(';',c):
            if '?' in each:
                each = each.split('?')[0]
            list_electyear_unique.append(each)
    elif ',' in str(e):
        c = e.count(',')
        for each in e.split(',',c):
            list_electyear_unique.append(each)
    elif 'x' or 'None' in str(e):
        continue
    elif '?' in str(e):
        list_electyear_unique.append(e.split('?')[0])
    else:
        list_electyear_unique.append(str(e))

#remove white space
list_electyear_unique = [item.strip() for item in list_electyear_unique]
list_electyear_unique_final = list(set(list_electyear_unique))


         

In [8]:
list_electyear_unique_final.sort()
del list_electyear_unique_final[0]

for year in list_electyear_unique_final:
    df[year] = df['re-elect years'].apply(lambda x: 1 if (year in str(x) and '?' not in str(x)) else 0)
del df['re-elect years']

In [9]:
df = df.drop([658,659,660,661])
df['grad year'] = df['grad year'].apply(lambda x: x.split(',')[0] if ',' in str(x) else x)
df['notes'].unique()

array([nan, 'removed from office in 2009', 'left office in 2009',
       'lost re-election', 'retired in 2011',
       'appointed to appellate dept in 2014',
       'contested in 2012, retired in 2015', 'lost re-election in 2004',
       'retired in 2012', 'retired in 2015', 'retired in 2009',
       'died in 2016', 'retired in 2005; died in 2015',
       'retired/ended in 2012', 'died in 2015', 'died in 2010',
       'retired in 2008', 'retired in 2013',
       'retired in 2010; died in 2016', 'retired in 2014',
       'retired in 2010', 'appointed to appellate dept in 2010',
       'appointed to appellate dept in 2008',
       'appointed to inspector general in 2008', 'retired in 2006',
       'arrested in 2007', 'appointed to US District Court',
       'left in 2011 claiming low compensation', 'lost re-election bid',
       2010, 'lost re-election, no endorsement', 'retired in 2007',
       'appointed to appellate dept', 'died in 2006',
       'appoint to appellate dept 2002', 'reti

In [10]:
notes_column = ['retired','arrested','lost','died','left','appointed','contested']
for notes in notes_column:
    df[notes] = df['notes'].apply(lambda x: 1 if notes in str(x) else 0)
df = df.rename(columns={'lost': 'lost-reelection', 'appointed': 'appointed-to-other-depart'})
del df['notes']

In [11]:
delete_columns = ['Unnamed: 28','web','lower only','no experience info']
for de in delete_columns:
    del df[de]

In [12]:
remove_list = ['missing','x','?',0]
df['start year'] = df['start year'].apply(lambda x:'Unknown' if (x in remove_list or '?' in str(x)) else x)

In [13]:
df_1 = df[df['multiple'] != 1]
df_2 = df[df['multiple'] == 1]

In [14]:
columns_to_delete_1 = ['judges (new)', 'Unnamed: 1','      Freq.     Percent        Cum.','multiple','misspell','missing']
for col in columns_to_delete_1:
    del df_1[col]
columns_to_delete_2 = ['      Freq.     Percent        Cum.','multiple','misspell','missing']
for col in columns_to_delete_2:
    del df_2[col]

In [15]:
df_2 = df_2.rename(columns={'judges (original)': 'judges_last_name', 'Unnamed: 3': 'judges_first_name'})

In [16]:
df_1 = df_1.rename(columns={'judges (original)': 'judges_last_name', 'Unnamed: 3': 'judges_first_name'})

In [17]:
del df_2['Unnamed: 1']
del df_2['judges (new)']

In [18]:
df_final = df_1.append(df_2)

In [19]:
df_final

,judges_last_name,judges_first_name,law school,grad year,elect,military,ADA,APD,Prof,female,...,2014,2015,2016,retired,arrested,lost-reelection,died,left,appointed-to-other-depart,contested
0,aarons,NaN,cuny,1989,0.0,0.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,abdus-salaam,NaN,columbia,1977,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,adams,NaN,tulsa,1977,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,adlerberg,NaN,brooklyn,1959,NaN,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
6,affronti,NaN,sanfrancisco,1963,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,alessandro,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
8,alfieri,NaN,pace,1979,0.0,0.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,allen,NaN,cornell,1974,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
10,aloi,NaN,akron,1970,NaN,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
11,aloise,NaN,st.johns,1982,NaN,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
df_final.to_csv('judges_final.csv')

    Cleaning Appellate Table

In [54]:
#read sheet from excel file
df_appellate = pd.read_excel(datadir+'/judges.xlsx', sheetname="appellate")

In [55]:
#reset index and change column name for last name
df_appellate = df_appellate.reset_index()
df_appellate = df_appellate.rename(columns={'index':'Appellate_Last_Name'})

In [56]:
#clean law school based on same format as judges
df_appellate['law school']= df_appellate['law school'].fillna('Unknown')
df_appellate['law school'] = df_appellate['law school'].apply(lambda x:re.sub(' ','',x.strip()).lower())

In [57]:
df_appellate['law school'].unique()

array(['buffalo', 'vermont', 'tennessee', 'whittier', 'st.johns',
       'syracuse', 'unknown', 'georgetown', 'harvard', 'albany', 'cornell'], dtype=object)

In [58]:
df_appellate['grad year'] = df_appellate['grad year'].apply(lambda x: x if pd.isnull(x) else int(x))

In [70]:
list_electyear_appellate = df_appellate['re-elect years'].unique()

In [80]:
list_electyear_unique = []
for e in list_electyear_appellate:
    if ',' in str(e):
        c = e.count(',')
        for each in e.split(',',c):
            list_electyear_unique.append(each)
    elif 'x' in str(e):
        continue
    elif '?' in str(e):
        list_electyear_unique.append(e.split('?')[0])
    else:
        list_electyear_unique.append(str(e))

list_electyear_unique_2 = []
for e in list_electyear_unique:
    if '?' in str(e):
        list_electyear_unique_2.append(e.split('?')[0])
    else:
        list_electyear_unique_2.append(str(e))
        
    
#remove white space
list_electyear_unique_2 = [item.strip() for item in list_electyear_unique_2]
list_electyear_unique_final = list(set(list_electyear_unique_2))

In [85]:
for year in list_electyear_unique_final:
    df_appellate[year] = df_appellate['re-elect years'].apply(lambda x: 1 if (year in str(x) and '?' not in str(x)) else 0)

In [87]:
del df_appellate['re-elect years']

In [90]:
#deal with notes column
df_appellate['notes'].unique()

array(['nomintated to Court of Appeals in Jan 2015', nan,
       'presiding justice', 'retired in 2013', 'retired in 2012',
       'retired in 2011', 'retired in 2010', 'retired 2009',
       'returned to practice, 2009', 'retired 2006',
       'presiding justice, appointed Court of Appeals 2006',
       'retired in 2006', 'retired in 2005', 'retired in 2004'], dtype=object)

In [91]:
notes_column = ['retired','presiding justice']
for notes in notes_column:
    df_appellate[notes] = df_appellate['notes'].apply(lambda x: 1 if notes in str(x) else 0)

del df_appellate['notes']

In [92]:
df_appellate

,Appellate_Last_Name,law school,grad year,elect,military,ADA,APD,Prof,female,no experience info,...,2006,2007,2008,2010,2011,2013,2014,2015,retired,presiding justice
0,fahey,buffalo,1984.0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,smith,vermont,NaN,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,scudder,tennessee,NaN,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,centra,buffalo,1983.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,peradotto,buffalo,1984.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,whalen,buffalo,1983.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,sconiers,buffalo,1973.0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
7,carni,whittier,1985.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,lindley,buffalo,1989.0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,valentino,st.johns,1972.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
df_appellate.to_csv('appellate_final.csv')